In [ ]:
import Thymio as th
import time
import costmap_path as cp
import cv2
import numpy as np

#FIXME: not right original image
original_image = cv2.imread('grid.png',0)
costmap, block_size, start, goal, display_image =cp.init(original_image)
path,costmap = cp.update(costmap, start, goal, display_image,[])
next_goal = path[0]
current_pos = np.array([0,0,0])
current_speed = np.array([0,0,0])
camera_position = (0,0,0)
z_predicted = np.concatenate([current_pos,current_speed]) #kalman_state
sigma_predicted = np.diag([0.1,0.1,0.1,0,0,0])
thymio = th.Thymio()
await thymio.iniateLock()

camera_on = True
while True:
    next_goal = path[0]
    thymio.navigate(current_pos, next_goal)
    current_pos = z_predicted[:2] # Is used for the controller

    z_filtered, sigma_filtered= thymio.filtering_step(z_k_k_1 = z_predicted, 
                                    sigma_k_k_1 = sigma_predicted, 
                                    V_left_measure = thymio.getWheelL(), 
                                    V_right_measure = thymio.getWheelR(), 
                                    camera_working=camera_on, 
                                    x_measured=camera_position[0], 
                                    y_measured=camera_position[1],
                                    theta_measured= camera_position[2])
    
    z_predicted, sigma_predicted = thymio.prediction_step(z_filtered, sigma_filtered)

    obstacle,detected = th.getObstaclePosition()
    if detected:
        print("Obstacle detected")
        #TODO: use the obstacle position to update the costmap
        path,costmap = cp.update(costmap, current_pos, goal, display_image,obstacle)
        
    if thymio.robot_close_waypoint(current_pos[:1], next_goal[0], next_goal[1]):
        path.pop(0)
        if len(path) == 0:
            print("Goal reached")
            break
        
        
    
    
    
    
    
    
    
    
 







AttributeError: 'NoneType' object has no attribute 'shape'